In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import scanpy as sc

from IPython.display import display

/public/workspace/licanchengup/apps/miniconda3/envs/csMAHN/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sys
p_link = Path('/public/workspace/licanchengup/link')
p_test = p_link.joinpath('test')
p_root = Path('/public/workspace/licanchengup/link/db/came/dataset/Retina')
None if str(p_test) in sys.path else sys.path.append(str(p_test))

In [3]:
from func import h5ad_to_mtx, group_agg


[func]----------------------------------
group_agg
rm_rf
get_path_varmap
get_type_counts_info		aligned
aligned_type			aligned
test_came_and_csMAHN
h5ad_to_mtx
load_adata

[result tools]--------------------------
get_test_result_df
show_umap
get_res_obs
get_publi_head

[result tools][re plot umap]-----------
get_df_color			Convenient
> [dssp umap]
export_legend
dssp_plot_umap_legend_sp	Convenient
dssp_plot_umap_legend_cell_type	Convenient
dssp_plot_umap
dssp_get_umap_name
dssp_plot_save_umap		Convenient
> [umap]
get_default_df_color
plot_umap
get_umap_name
plot_save_umap			Convenient
----------------------------------------
    


In [4]:
adata = sc.read_h5ad('Macosko.h5ad')
display(
    adata,
    adata.obs.head(2),
    adata.obs.shape,
    adata.var.head(2),
    adata.var.shape)

AnnData object with n_obs × n_vars = 44808 × 23288
    obs: 'cell_type1', 'cluster_id', 'organism', 'dataset_name', 'platform', 'organ', 'data_type', 'cell_ontology_class', 'cell_ontology_id', 'n_genes', 'n_counts', '__libsize__'
    var: 'variable_genes'
    uns: 'cell_ontology_class_colors', 'cell_type1_colors', 'data_quality', 'neighbors', 'umap', 'variable_genes'
    obsm: 'X_umap', 'latent'
    obsp: 'connectivities', 'distances'

,cell_type1,cluster_id,organism,dataset_name,platform,organ,data_type,cell_ontology_class,cell_ontology_id,n_genes,n_counts,__libsize__
cell_id,,,,,,,,,,,,
p1_AAAAAAACAGTC,muller,34,Mus musculus,Macosko,Drop-seq,Retina,raw,Muller cell,CL:0011107,1243,1864,1864.0
p1_AAAAAAATGGTA,rods,24,Mus musculus,Macosko,Drop-seq,Retina,raw,retinal rod cell,CL:0000604,488,818,818.0


(44808, 12)

,variable_genes
0610005C13Rik,False
0610007P14Rik,False


(23288, 1)

是count, perfectly

In [5]:
_temp = adata.X[:100, :100]
display(np.ravel(_temp[_temp > 0])[:10])
del _temp

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [6]:
print(*adata.obs.columns, sep='\n')
display(adata.obs.head(2))

cell_type1
cluster_id
organism
dataset_name
platform
organ
data_type
cell_ontology_class
cell_ontology_id
n_genes
n_counts
__libsize__


,cell_type1,cluster_id,organism,dataset_name,platform,organ,data_type,cell_ontology_class,cell_ontology_id,n_genes,n_counts,__libsize__
cell_id,,,,,,,,,,,,
p1_AAAAAAACAGTC,muller,34,Mus musculus,Macosko,Drop-seq,Retina,raw,Muller cell,CL:0011107,1243,1864,1864.0
p1_AAAAAAATGGTA,rods,24,Mus musculus,Macosko,Drop-seq,Retina,raw,retinal rod cell,CL:0000604,488,818,818.0


In [7]:
for _ in 'cell_type1,cell_ontology_class'.split(','):
    print(_.ljust(50, '-'))
    display(adata.obs[_].value_counts(dropna=False))

cell_type1----------------------------------------


cell_type1
rods                    29400
bipolar                  6285
amacrine                 4426
cones                    1868
muller                   1624
ganglion                  432
horizontal                252
vascular_endothelium      252
fibroblasts                85
microglia                  67
pericytes                  63
astrocytes                 54
Name: count, dtype: int64

cell_ontology_class-------------------------------


cell_ontology_class
retinal rod cell                 29400
retinal bipolar neuron            6285
amacrine cell                     4426
retinal cone cell                 1868
Muller cell                       1624
retinal ganglion cell              432
blood vessel endothelial cell      252
retina horizontal cell             252
fibroblast                          85
microglial cell                     67
pericyte cell                       63
astrocyte                           54
Name: count, dtype: int64

In [8]:
adata.obs = adata.obs.loc[:, 'cell_type1,cell_ontology_class'.split(',')]
adata.obs['pre_ct'] = adata.obs['cell_type1']
adata.obs['cell_type'] = adata.obs['cell_ontology_class'].map( lambda k:
{
    'amacrine cell': 'AC',
    'blood vessel endothelial cell': 'V/E cell',
    'Muller cell': 'Macroglial cell',
    'microglial cell': 'Microglia cell',
    'pericyte cell': 'Pericyte cell',
    
    'retina horizontal cell': 'HC',
    'retinal bipolar neuron': 'BC',
    'retinal cone cell': 'Cones',
    
    'retinal rod cell': 'Rods',
    'retinal ganglion cell': 'RGC',
    # ?
    'astrocyte': 'astrocyte',
    'fibroblast': 'fibroblast'
}.setdefault(k,k))
display(adata.obs.head(2), adata.obs.shape)
h5ad_to_mtx(adata, p_root.joinpath('Macosko'))

,cell_type1,cell_ontology_class,pre_ct,cell_type
cell_id,,,,
p1_AAAAAAACAGTC,muller,Muller cell,muller,Macroglial cell
p1_AAAAAAATGGTA,rods,retinal rod cell,rods,Rods


(44808, 4)

frist 10 data.X nonzero elements:
 [[1 1 1 1 1 1 1 1 1 1]]
[out] /public/workspace/licanchengup/link/db/came/dataset/Retina/Macosko


In [9]:

# {k:'' for k in adata.obs['cell_ontology_class'].unique()}